In [1]:
import numpy as np
from keras import layers
from tensorflow.nn import avg_pool2d, max_pool2d
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
import numpy as np
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import TensorBoard
#from tensorflow.keras.applications import ResNet50
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.
C:\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [98]:

def identity_block(X, f, filters, stage, block):
 
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [99]:
def convolutional_block(X, f, filters, stage, block, s = 2):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X

 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)


    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)


    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    
    return X

In [100]:
def ResNet50(input_shape = (224, 224, 3), classes = 2):
  
    X_input = Input(input_shape)

    
    X = ZeroPadding2D((3, 3))(X_input)
    
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    X = Flatten()(X)
    X = Dense(256, activation='relu')(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [101]:
res = ResNet50(input_shape = (128, 128, 3), classes = 2)
model=Sequential()
model.add(ResNet50(input_shape = (128, 128, 3), classes = 2))


In [102]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                   metrics = ['accuracy'])
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_6 (ZeroPadding2D (None, 134, 134, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        zero_padding2d_6[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

SyntaxError: can't assign to operator (<ipython-input-111-1a919625dfe2>, line 1)

In [3]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
train= train_datagen.flow_from_directory('C:/Users/G5/Desktop/DATASET/TRAIN',
                                                 target_size = (128, 128),
                                                 batch_size = 22564,
                                                 class_mode = 'binary')
test=train_datagen.flow_from_directory('C:/Users/G5/Desktop/DATASET/TEST',
                                                 target_size = (128, 128),
                                                 batch_size = 2513,
                                                 class_mode = 'binary')
X_train_orig,Y_train_orig=train.next()
X_test_orig,Y_test_orig=test.next()

Y_train_orig=Y_train_orig.astype(int)
Y_test_orig=Y_test_orig.astype(int)

#X_train_orig= keras.utils.to_categorical(X_train_orig, NB_CLASSES)
#X_test_orig = keras.utils.to_categorical(X_test_orig, NB_CLASSES)

Y_train = convert_to_one_hot(Y_train_orig, 2).T
Y_test = convert_to_one_hot(Y_test_orig, 2).T

print(X_train_orig.shape)
print(X_test_orig.shape)
print(Y_train.shape)
print(Y_test.shape)

print(Y_test)


Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.
(22564, 128, 128, 3)
(2513, 128, 128, 3)
(22564, 2)
(2513, 2)
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [9]:
print(X_train_orig)

[[[[0.5882353  0.60784316 0.61960787]
   [0.13333334 0.09803922 0.07843138]
   [0.09803922 0.13725491 0.14509805]
   ...
   [0.9960785  0.97647065 0.5058824 ]
   [0.9960785  0.96470594 0.09019608]
   [1.         0.9333334  0.07450981]]

  [[0.5803922  0.6        0.6117647 ]
   [0.14509805 0.10980393 0.09019608]
   [0.10196079 0.14117648 0.14901961]
   ...
   [0.96470594 0.95294124 0.43921572]
   [0.9725491  0.95294124 0.01568628]
   [1.         0.94117653 0.00784314]]

  [[0.57254905 0.5921569  0.6039216 ]
   [0.16470589 0.12941177 0.10980393]
   [0.11764707 0.15686275 0.16470589]
   ...
   [0.9450981  0.92549026 0.46274513]
   [0.96470594 0.93725497 0.0509804 ]
   [1.         0.9294118  0.03921569]]

  ...

  [[0.59607846 0.5882353  0.5921569 ]
   [0.16862746 0.14509805 0.15294118]
   [0.14509805 0.10196079 0.11764707]
   ...
   [0.21960786 0.2901961  0.32941177]
   [0.20784315 0.2784314  0.31764707]
   [0.21176472 0.28235295 0.32156864]]

  [[0.6        0.6        0.6       ]
   [0.1

In [8]:
import tensorflow as tf
tf.keras.preprocessing.image.array_to_img(X_test_orig[2]).show()

In [105]:
model.fit(X_train_orig, Y_train, epochs = 5, batch_size = 32)

Epoch 1/5
22564/22564 [==============================] - 227s 10ms/step - loss: 8.5459 - accuracy: 0.4437
Epoch 2/5
22564/22564 [==============================] - 222s 10ms/step - loss: 8.5640 - accuracy: 0.4431
Epoch 3/5
22564/22564 [==============================] - 222s 10ms/step - loss: 8.5640 - accuracy: 0.4431
Epoch 4/5
22564/22564 [==============================] - 222s 10ms/step - loss: 8.5640 - accuracy: 0.4431
Epoch 5/5
22564/22564 [==============================] - 222s 10ms/step - loss: 8.5640 - accuracy: 0.4431


In [27]:
model.fit(X_train_orig, Y_train, epochs = 53, batch_size = 32)

Epoch 1/53
22564/22564 [==============================] - 110s 5ms/step - loss: 0.3362 - accuracy: 0.8608
Epoch 2/53
22564/22564 [==============================] - 110s 5ms/step - loss: 0.3248 - accuracy: 0.8654
Epoch 3/53
22564/22564 [==============================] - 110s 5ms/step - loss: 0.3195 - accuracy: 0.8678
Epoch 4/53
22564/22564 [==============================] - 110s 5ms/step - loss: 0.3066 - accuracy: 0.8716
Epoch 5/53
22564/22564 [==============================] - 110s 5ms/step - loss: 0.2923 - accuracy: 0.8803
Epoch 6/53
22564/22564 [==============================] - 110s 5ms/step - loss: 0.2832 - accuracy: 0.8875
Epoch 7/53
22564/22564 [==============================] - 110s 5ms/step - loss: 0.2749 - accuracy: 0.8886
Epoch 8/53
22564/22564 [==============================] - 110s 5ms/step - loss: 0.2617 - accuracy: 0.8954
Epoch 9/53
22564/22564 [==============================] - 110s 5ms/step - loss: 0.2515 - accuracy: 0.9003
Epoch 10/53
22564/22564 [=====================

In [28]:
print("hello")

hello


In [29]:
model.evaluate(X_test_orig,Y_test)

2513/2513 [==============================] - 4s 1ms/step


[0.7449393519546897, 0.870274543762207]

In [30]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 70, 70, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [33]:
pip install graphviz

  Using cached https://files.pythonhosted.org/packages/94/cd/7b37f2b658995033879719e1ea4c9f171bf7a14c16b79220bd19f9eda3fe/graphviz-0.13-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [35]:
pwd

'C:\\Users\\G5'

In [36]:
from tensorflow.python.keras.callbacks import TensorBoard

In [37]:
model.get_weights()

[array([[[[-2.47575045e-01,  4.24000956e-02,  1.79305196e-01, ...,
            3.80448312e-01,  1.77323699e-01, -1.84233204e-01],
          [-1.10277541e-01, -3.85526754e-02,  2.41256773e-01, ...,
            4.22509581e-01,  1.31325126e-01, -2.51242779e-02],
          [ 9.95227620e-02,  7.79459774e-02,  1.69216469e-01, ...,
            1.52625874e-01, -9.76761207e-02,  2.16462582e-01]],
 
         [[-2.22073004e-01,  4.74295244e-02,  8.63761306e-02, ...,
            5.83645821e-01, -1.99647933e-01, -3.20622623e-01],
          [-1.57085121e-01, -2.25733325e-01,  4.87674810e-02, ...,
            3.96697134e-01, -2.05664605e-01, -3.28162573e-02],
          [-2.77792588e-02, -4.35848869e-02, -2.69715749e-02, ...,
            8.89889374e-02, -2.97309726e-01,  1.63643792e-01]],
 
         [[-2.36625895e-01, -1.22478530e-01, -4.95584160e-02, ...,
            4.97888066e-02,  1.43663138e-01, -2.85108745e-01],
          [-1.76714927e-01, -5.13912022e-01, -6.95585534e-02, ...,
           -8.662

In [38]:
model.save('waste_segregation.h5')

In [39]:
from keras.models import load_model

In [40]:
new_model=load_model('waste_segregation.h5')

In [41]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [83]:
from PIL import Image
image = Image.open('C:/Users/G5/Desktop/O_1.jpg')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
# show the image
image.show()

JPEG
RGB
(208, 242)


In [3]:
import tensorflow as tf
img_array=tf.keras.preprocessing.image.img_to_array(image)
# print(img_array)
print(img_array)
img_array=img_array/255.
print(img_array)
img_array.resize([64,64,3])
print(img_array.shape)
img2=tf.keras.preprocessing.image.array_to_img(img_array)
img2.show()
print(img2.mode)

TypeError: float() argument must be a string or a number, not 'module'

In [85]:
inputttt = []
inputttt.append([img_array])


In [86]:
r=new_model.predict(inputttt)

In [87]:
print(r*100)

[[96.469246   3.5307481]]


In [1]:
model.summary()


NameError: name 'model' is not defined

In [4]:
model=load_model('waste_segregation.h5')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [25]:
from PIL import Image
image = Image.open('C:/Users/G5/Desktop/glass.jpg')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
# show the image
image.show()

JPEG
RGB
(300, 277)


In [26]:
import tensorflow as tf
img_array=tf.keras.preprocessing.image.img_to_array(image)
# print(img_array)
#print(img_array)
img_array=img_array/255.
#print(img_array)
img_array.resize([64,64,3])
print(img_array.shape)
img2=tf.keras.preprocessing.image.array_to_img(img_array)
img2.show()
print(img2.mode)

[[[0.19215687 0.1882353  0.18039216]
  [0.20392157 0.2        0.19215687]
  [0.22745098 0.22352941 0.21568628]
  ...
  [0.14509805 0.14117648 0.12156863]
  [0.13725491 0.13333334 0.11372549]
  [0.14117648 0.13725491 0.11764706]]

 [[0.19607843 0.19215687 0.18431373]
  [0.2        0.19607843 0.1882353 ]
  [0.21960784 0.21568628 0.20784314]
  ...
  [0.14509805 0.14901961 0.1254902 ]
  [0.14117648 0.13725491 0.11764706]
  [0.14117648 0.13725491 0.12156863]]

 [[0.20784314 0.20392157 0.19607843]
  [0.2        0.19607843 0.1882353 ]
  [0.20784314 0.20392157 0.19607843]
  ...
  [0.14117648 0.14509805 0.1254902 ]
  [0.13725491 0.14117648 0.12156863]
  [0.13725491 0.14117648 0.12156863]]

 ...

 [[0.3764706  0.37254903 0.3647059 ]
  [0.3882353  0.39215687 0.37254903]
  [0.36862746 0.3764706  0.35686275]
  ...
  [0.28627452 0.30980393 0.27058825]
  [0.2901961  0.3019608  0.27450982]
  [0.28235295 0.29411766 0.26666668]]

 [[0.36862746 0.3647059  0.35686275]
  [0.38431373 0.3882353  0.36862746]


In [27]:
inputttt = []
inputttt.append([img_array])

In [28]:
model.predict(inputttt)

array([[0.00905543, 0.9909445 ]], dtype=float32)

In [3]:
pwd

'C:\\Users\\G5'

In [15]:
new_model=load_model('model_2242.h5')

ValueError: ('Invalid `distribution` argument: expected one of {"normal", "uniform"} but got', 'truncated_normal')

In [1]:
import tensorflow as tf

C:\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [3]:
new_model=tf.keras.models.load_model(
    "D:\colab_model.h5",
    custom_objects=None,
    compile=True
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1000)              25636712  
_________________________________________________________________
flatten (Flatten)            (None, 1000)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               256256    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 25,893,482
Trainable params: 25,840,362
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
tf.test.is_gpu_available(
    cuda_only=True,
    min_cuda_compute_capability=None
)

In [8]:
import tensorflow as tf
import h5py


In [32]:
new_model=tf.keras.models.load_model(
    "D:/colab_model.h5",
    custom_objects=None,
    compile=True
)

In [5]:
from PIL import Image
image = Image.open('lal2.jpg')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
# show the image
#image.show()
#size = 224, 224
#image.thumbnail(size, Image.ANTIALIAS)

JPEG
RGB
(224, 224)


In [6]:
import matplotlib.pyplot as plt
import tensorflow as tf
img_array=tf.keras.preprocessing.image.img_to_array(image)
# print(img_array)
#print(img_array)
img_array=img_array/255.
#print(img_array)
#img_array.resize([224,224,3])
print(img_array.shape)
#img2=tf.keras.preprocessing.image.array_to_img(img_array)
plt.imshow(img_array)
#print(img2.mode)

(224, 224, 3)


In [16]:
inputt=[]
inputt.append([img_array])
new_model.predict(inputt)

UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node resnet50/conv1/Conv2D}}]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node resnet50/conv1/Conv2D}}]]
	 [[dense_1/Softmax/_5465]]
0 successful operations.
0 derived errors ignored.

In [11]:
from tensorflow.keras import Model